In [1]:
import stanza
import urduhack

import pandas as pd
import re
import numpy as np
from urduhack import normalize
from urduhack.preprocessing import normalize_whitespace
from urduhack.preprocessing import remove_punctuation
from urduhack.preprocessing import replace_urls
from urduhack.preprocessing import replace_emails
from urduhack.preprocessing import replace_numbers
from urduhack.preprocessing import remove_english_alphabets

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances


from bs4 import BeautifulSoup
from scipy.stats import multivariate_normal as mvn

import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import string

D:\Softwares\Anaconda\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
df = pd.read_csv('tweets.csv',encoding='utf-8',error_bad_lines=False)

D:\Softwares\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%a %b %d %H:%M:%S +0000 %Y')

# Pre-Processing

In [4]:
nlp = stanza.Pipeline(lang='ur', processors='tokenize,lemma')

2022-04-12 17:46:10 INFO: Loading these models for language: ur (Urdu):
| Processor | Package |
-----------------------
| tokenize  | udtb    |
| lemma     | udtb    |

2022-04-12 17:46:10 INFO: Use device: cpu
2022-04-12 17:46:10 INFO: Loading: tokenize
2022-04-12 17:46:10 INFO: Loading: lemma
2022-04-12 17:46:10 INFO: Done loading processors!


In [ ]:
# urdu_swords = "ایت نہیں نے ہاں ہر ہم ہمارا ہمارے ہماری ہو ہوا ہوتا ہوتے ہوتی ہوتیں ہوں ہونا ہونگے ہونے ہونی ہوئے ہوئی ہوئیں ہے ہی ہیں و والا والوں والے والی وہ وہاں وہی وہیں یا یعنی یہ یہاں یہی یہیںاب ابھی اپنا اپنے اپنی اٹھا اس اسے اسی اگر ان انہوں انہی انہیں انھیں او اور اے ایسا ایسے ایسی ایک آ آپ آتا آتے آتی آگے آنا آنے آنی آئے آئی آئیں آیا با بڑا بڑے بڑی بعد بعض بلکہ بہت بھی بے پاس پر پہلے پھر تا تاکہ تب تجھ تجھے تک تم تمام تمہارا تمہارے تمھارے تمہاری تمہیں تمھیں تھا تھے تھی تھیں تو تیری تیرے جا جاتا جاتی جاتے جاتی جانے جانی جاؤ جائے جائیں جب جس جن جنہوں جنہیں جو جیسا جیسے جیسی جیسوں چاہیئے چلا چاہے چونکہ حالاں حالانکہ دو دونوں دوں دے دی دیا دیں دیے دیتا دیتے دیتی دینا دینے دینی دیئے ڈالا ڈالنا ڈالنے ڈالنی ڈالے ڈالی ذرا رکھا رکھتا رکھتے رکھتی رکھنا رکھنے رکھنی رکھے رکھی رہ رہا رہتا رہتے رہتی رہنا رہنے رہنی رہو رہے رہی رہیں زیادہ سا سامنے سب سکتا سو سے سی شاید صرف طرح طرف عین کا کبھی کچھ کہہ کر کرتا کرتے کرتی کرنا کرنے کرو کروں کرے کریں کس کسے کسی کہ کہا کہے کو کون کوئی کے کی کیا کیسے کیوں کیونکہ کیے کئے گا گویا گے گی گیا گئے گئی لا لاتا لاتے لاتی لانا لانے لانی لایا لائے لائی لگا لگے لگی لگیں لو لے لی لیا لیتا لیتے لیتی لیکن لیں لیے لئے مجھ مجھے مگر میرا میرے میری میں نا نہ نہایت نہیں نے ہاں ہر ہم ہمارا ہمارے ہماری ہو ہوا ہوتا ہوتے ہوتی ہوتیں ہوں ہونا ہونگے ہونے ہونی ہوئے ہوئی ہوئیں ہے ہی ہیں و والا والوں والے والی وہ وہاں وہی وہیں یا یعنی یہ یہاں یہی یہیں"
# stop_words = ""
# doc = nlp(urdu_swords)
# a = doc.to_dict()
# a=a[0]
# for i in a:
#     stop_words=stop_words+(i['lemma'])+" "
    
# #print(stop_words)

# stop_words = stop_words.split()
# stop_words = " ".join(sorted(set(stop_words), key=stop_words.index))


In [5]:
stop_word = 'ایت نہیں نے ہاں ہر ہم مےں ہو ہے ہی و والا وہ وہاں وہی وہیں یا یعنی یہ یہاں یہیںاب ابھی اپنا اٹھ اگر او اور اے ایسا ایک آ آپ آگے آنی آئی با بڑا بڑی بعد بعض بلکہ بہت بھی بے پاس پر پہلے پھر تا تاکہ تب تجھ تو تک تمام تمہارا تمھارا تم تمھ تھا تیری تیرا جا جاؤ جب جو جیسا جیسے جیسی جیس چاہیئے چل چاہ چونکہ حالاں حالانکہ دو دوں دے دینی ڈال ڈالنا ڈالنی ذرا رکھ رکھنی رہ رہنی زیادہ سا سامنے سب سک سو سے سی شاید صرف طرح طرف عین کا کبھی کچھ کہہ کر کون کوا کوئی کہ کہا کو کیسا کیوں کیونکہ گا گویا گے گی لا لاتا لاتی لانی لائے لگ لو لے لیکن لیے لئے مجھ مگر میرا میرے میں نا نہ نہایت یہیں'
stop_word = stop_word.split()

In [6]:
def preprocess(word):
    word = re.sub("\W", " ",word)
    word = re.sub("[a-zA-Z0-9]","",word)
    word = normalize_whitespace(word)
    word = remove_punctuation(word)
    word = replace_urls(word)
    word = replace_emails(word)
    #word = normalize(word)
    word = replace_numbers(word)
    #STanza lemmentization normalize
    
    doc = nlp(word)
    a = doc.to_dict()
    if(len(a) != 0):
        a = a[0]
        word = ''
        for i in a:
            if not i['lemma'] in stop_word:
                if(i['id']==(a[-1]['id'])):
                    word=word+i['lemma']
                else:
                    word = word+i['lemma']+' '
    
    return word

In [7]:
def remove_duplicate_words(string):
        x = string.split()
        x = sorted(set(x), key = x.index)
        return ' '.join(x)

In [22]:
df[1:100]

,Unnamed: 0,tweets,timestamp
1,1,"RT @SSEHBAI1: لوگ باہر: جیسے لوگ نکل رہے ہیں, ...",2022-04-10 20:09:56
2,2,RT @FaisalJavedKhan: آپکی رائے میں کیا تحریک ا...,2022-04-10 20:09:57
3,3,RT @Abdulla78487680: جس سے جگر لالہ میں ٹھنڈک ...,2022-04-10 20:09:57
4,4,RT @fawadchaudhry: شہباز شریف کے کاغذات نامزدگ...,2022-04-10 20:09:57
5,5,RT @Sarfraz053: @MaddyWithKhan عمران خان نے کہ...,2022-04-10 20:09:57
...,...,...,...
95,95,اب شاہین کا تجربہ کرو یا مسرت شاہین کا\r\nاگلے...,2022-04-10 20:10:03
96,96,RT @MishalBukhari5: شریک جرم ہی ہو گا جو اب رہ...,2022-04-10 20:10:03
97,97,RT @ImranRiazKhan: ان سب کی مرمت ہو رہی ہے,2022-04-10 20:10:03
98,98,RT @SMGhouri2: @SSEHBAI1 ایک بات تو طے ہے کہ ل...,2022-04-10 20:10:03


In [9]:
list1 = []
for i in range(len(df.index)):
    sent = df[df.columns[1]][i]
    list1.append(sent)
    

In [10]:
sent_p =[]
for tweet in list1:
    sent_p.append(preprocess(tweet))

# Timestamp

In [11]:
df2 = pd.DataFrame(sent_p,columns = ["tweets"])

In [12]:
df2['timestamp'] = pd.to_datetime(df['timestamp'], format='%a %b %d %H:%M:%S +0000 %Y')

In [18]:
def timefilter(start,end,dft):
    filteredTw = []
    filteredTm = []
    for i in range(0,len(dft)):
        if (dft['timestamp'][i] >= start) & (dft['timestamp'][i] <= end):
            filteredTw.append(dft['tweets'][i])
            filteredTm.append(dft['timestamp'][i])
    filtered = pd.DataFrame()
    if(filteredTm and filteredTw is not None):
        filtered['tweets'] = pd.DataFrame(filteredTw)
        filtered['timestamp'] = pd.DataFrame(filteredTm)
    else:
        print("No values found, returning default")
        return dft
    
#     filtered = dft[(dft['timestamp'] >= start) & (dft['timestamp'] <= end)]
    return filtered

In [23]:
splitdf2 = timefilter(pd.Timestamp(2022,4,10,20,9,5),pd.Timestamp(2022,4,10,20,9,59),df2)
splitdf = timefilter(pd.Timestamp(2022,4,10,20,9,5),pd.Timestamp(2022,4,10,20,9,59),df)


In [24]:
splitdf

,tweets,timestamp
0,@SaeedGhani1 چاچا\r\nستو پی لیا ہو تو دیکھو\r\...,2022-04-10 20:09:56
1,"RT @SSEHBAI1: لوگ باہر: جیسے لوگ نکل رہے ہیں, ...",2022-04-10 20:09:56
2,RT @FaisalJavedKhan: آپکی رائے میں کیا تحریک ا...,2022-04-10 20:09:57
3,RT @Abdulla78487680: جس سے جگر لالہ میں ٹھنڈک ...,2022-04-10 20:09:57
4,RT @fawadchaudhry: شہباز شریف کے کاغذات نامزدگ...,2022-04-10 20:09:57
5,RT @Sarfraz053: @MaddyWithKhan عمران خان نے کہ...,2022-04-10 20:09:57
6,پہلے والے غیر جمہوری اور غیر آئینی طریقے سے نک...,2022-04-10 20:09:57
7,RT @ImranRiazFan_: پاکستان کی تاریخ میں پہلی د...,2022-04-10 20:09:57
8,RT @haroon_natamam: راولپنڈی میں لوگ سڑکوں پہ ...,2022-04-10 20:09:57
9,"RT @SSEHBAI1: لوگ باہر: جیسے لوگ نکل رہے ہیں, ...",2022-04-10 20:09:57


# TF-IDF

In [ ]:
from sklearn.cluster import KMeans  
data = splitdf2
tf_idf_vectorizor = TfidfVectorizer(max_features = 5000)
tf_idf = tf_idf_vectorizor.fit_transform(splitdf2['tweets'][:])
tf_idf_norm = normalize(tf_idf)

tf_idf_array = tf_idf_norm.toarray()
a = pd.DataFrame(tf_idf_array, columns=tf_idf_vectorizor.get_feature_names())

# Kneed-Point

In [ ]:
df.dtypes

# Clustering with Time-stamp

In [ ]:
k = 3
model = KMeans(n_clusters=k)

labels = model.fit(tf_idf_array)

In [ ]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = tf_idf_vectorizor.get_feature_names()
for i in range(k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])

In [ ]:
sklearn_pca = PCA(n_components = 2)
Y_sklearn = sklearn_pca.fit_transform(tf_idf_array)
predicted_values = model.predict(tf_idf_array)

In [ ]:
plt.scatter(Y_sklearn[:, 0], Y_sklearn[:, 1], c=predicted_values, s=50, cmap='viridis')

In [ ]:
clusters = []
for i in range(k):
    clusters.append([])

for i,row in splitdf.iterrows():
    clusters[predicted_values[i]].append(row['tweets'])

In [ ]:
for i in range(len(clusters)):
    print("Cluster: " + str(i))
    for j in range(5):
        print(clusters[i][j]+'\n')